### Prepare Library and Dependencies

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
from ultralytics import YOLO
import cv2
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Training

In [ ]:
def crop_bag_from_folder(model_path, input_folder, save_folder, conf_threshold=0.45):
    # Load model
    model = YOLO(model_path)

    # Bikin folder output kalau belum ada
    os.makedirs(save_folder, exist_ok=True)

    # Predict semua gambar di folder
    results = model.predict(source=input_folder, conf=conf_threshold, save=False)

    # Loop tiap hasil prediksi
    for result in results:
        img_path = result.path  # path asli gambar
        img_name = os.path.basename(img_path)
        img = cv2.imread(img_path)

        # Ambil semua detection
        boxes = result.boxes.xyxy.cpu().numpy()  # xmin, ymin, xmax, ymax
        scores = result.boxes.conf.cpu().numpy()  # confidence

        if len(boxes) == 0:
            print(f"No detection for {img_name}. Skipping.")
            continue

        # Cari detection dengan confidence maksimal
        max_conf = scores.max()
        candidates_idx = [i for i, s in enumerate(scores) if s == max_conf]

        if len(candidates_idx) == 1:
            best_idx = candidates_idx[0]
        else:
            # Kalau ada lebih dari satu dengan confidence sama, pilih bbox terkecil
            areas = [(boxes[i][2] - boxes[i][0]) * (boxes[i][3] - boxes[i][1]) for i in candidates_idx]
            best_idx = candidates_idx[areas.index(min(areas))]

        # Ambil koordinat bbox terbaik
        xmin, ymin, xmax, ymax = boxes[best_idx].astype(int)

        # Crop gambar
        cropped_img = img[ymin:ymax, xmin:xmax]

        # Save crop
        save_path = os.path.join(save_folder, img_name)
        cv2.imwrite(save_path, cropped_img)

    print(f"Selesai crop semua gambar dari {input_folder} ke {save_folder}!")

In [ ]:
crop_bag_from_folder(
    model_path='/content/drive/MyDrive/dataset/best.pt',
    input_folder='/content/drive/MyDrive/dataset/images/stock_images/abnormal',
    save_folder='/content/drive/MyDrive/dataset/images/stock_images/cropped/abnormal',
    conf_threshold=0.45
)


image 1/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-12.jpg: 640x640 1 effluent dialysate bag, 2669.3ms
image 2/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-13.jpg: 640x640 1 effluent dialysate bag, 2394.5ms
image 3/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-155.jpg: 640x640 1 effluent dialysate bag, 2344.9ms
image 4/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-156.jpg: 640x640 1 effluent dialysate bag, 2747.2ms
image 5/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-21.jpg: 640x640 1 effluent dialysate bag, 3695.7ms
image 6/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-235.jpg: 640x640 1 effluent dialysate bag, 2594.1ms
image 7/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnormal-236.jpg: 640x640 1 effluent dialysate bag, 2396.6ms
image 8/46 /content/drive/MyDrive/dataset/images/stock_images/abnormal/abnorma

In [ ]:
crop_bag_from_folder(
    model_path='/content/drive/MyDrive/dataset/best.pt',
    input_folder='/content/drive/MyDrive/dataset/images/images_v2/normal',
    save_folder='/content/drive/MyDrive/dataset/images/images_v2/cropped/normal',
    conf_threshold=0.45
)


image 1/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-0.jpg: 640x640 1 effluent dialysate bag, 2800.7ms
image 2/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-1.jpg: 640x640 1 effluent dialysate bag, 2947.4ms
image 3/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-10.jpg: 640x640 1 effluent dialysate bag, 2128.7ms
image 4/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-100.jpg: 640x640 1 effluent dialysate bag, 3530.9ms
image 5/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-101.jpg: 640x640 1 effluent dialysate bag, 2344.0ms
image 6/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-102.jpg: 640x640 1 effluent dialysate bag, 3595.0ms
image 7/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-103.jpg: 640x640 1 effluent dialysate bag, 2872.3ms
image 8/342 /content/drive/MyDrive/dataset/images/images_v2/normal/normal-104.jpg: 640x640 1 effluent dialysate bag, 2377.